This code uses the SSJ environment to solve a simple 2-generation OLG model without bequests.

In [1]:

calibration = {'thetao': 0.1, 'thetay': 1.9, 'pi_o': 0.5, 'pi_y': 0.5, 'K': 1.6, 'L': 1, 'delta': 0.18749999999999997, 'H': 2, 'alpha': 0.3, 'sig_y': 1.4, 'nh': 0.9, 'gamma': 1.0, 'Tax': 0.5319999999999999, 'taul': 0.3, 'co': 1.7476602802635783, 'cy': 1.6233880298203545, 'ly': 1.9023585540600496, 'lo': 0.09764144761560888, 'ay': 1.1520000010308595, 'r': 0.05, 'w': 0.8866666666666665, 'Y': 1.2666666666666666, 'Z': 1.100085263739135, 'bequests': 0, 'NFA': 0.36}

import numpy as np

H=2
sigvec = np.arange(0,H,1)

from sequence_jacobian import simple, solved, create_model, combine


The calibration came from a separate steady state file. The firm block is off the shelf.

In [3]:
@simple
def firm(K, L, Z, alpha, delta):
    r = alpha * Z * (K / L) ** (alpha-1) - delta
    w = (1 - alpha) * Z * (K / L) ** alpha
    Y = Z * K ** alpha * L ** (1 - alpha)
    return r, w, Y

The 2 living generations take fiscal policy (taul, T), wages (w), and the interest rate (r) to solve for their lifetime budget constraint, and their FOC give their consumption when old. That is, I solve for co (the consumption of the current old) and co_y the consumption when old of the current young.

In [4]:
@simple
def lifetimebudget_y(co_y,w,r,beta,psil,Tax,taul,gamma,H,sig_y,nh,thetao,thetay):
    
    # EIS over the life-cycle
    sig = np.ones(H)*sig_y
    sig = sig*nh**sigvec
    
    cy_y = (beta*(1+r(+1))*co_y**(-sig[H-1]))**(-1/sig[0])
    
    ly_y = ((cy_y**(-sig[0])*(1-taul)*w*thetay)/psil)**(1/gamma)
    lo_y = ((co_y**(-sig[H-1])*(1-taul)*w(+1)*thetao)/psil)**(1/gamma)
    
    lifebud_y = (thetay*w*ly_y*(1-taul)+Tax-cy_y)*(1+r(+1))+Tax+thetao*w(+1)*lo_y*(1-taul)-co_y 
    
    return lifebud_y

@simple
def lifetimebudget_o(co,w,r,beta,psil,Tax,taul,gamma,H,sig_y,nh,thetay,thetao):
    
    # EIS over the life-cycle
    sig = np.ones(H)*sig_y
    sig = sig*nh**sigvec
    
    cy_o = (beta*(1+r)*co**(-sig[H-1]))**(-1/sig[0])
    
    ly_o = ((cy_o**(-sig[0])*(1-taul)*w(-1)*thetay)/psil)**(1/gamma)
    lo_o = ((co**(-sig[H-1])*(1-taul)*w*thetao)/psil)**(1/gamma)
    
    lifebud_o = (thetay*w(-1)*ly_o*(1-taul)+Tax-cy_o)*(1+r)+Tax+thetao*w*lo_o*(1-taul)-co
    
    return lifebud_o

@simple
def household(co,co_y,w,r,beta,psil,Tax,taul,gamma,H,sig_y,nh,thetay,thetao):
    
    # EIS over the life-cycle
    sig = np.ones(H)*sig_y
    sig = sig*nh**sigvec
    
    cy = (beta*(1+r(+1))*co_y**(-sig[H-1]))**(-1/sig[0])
    
    ly = ((cy**(-sig[0])*(1-taul)*w*thetay)/psil)**(1/gamma)
    
    ay = thetay*w*ly*(1-taul)+Tax-cy
    
    lo = ((co**(-sig[H-1])*(1-taul)*w*thetao)/psil)**(1/gamma)

    
    return ay, cy, ly, lo

households = combine([household,lifetimebudget_o,lifetimebudget_y])
household_solved = households.solved(unknowns={'co':.8, 'co_y': .8},targets=['lifebud_o', 'lifebud_y'],solver='broyden_custom')

print(f"Inputs: {household_solved.inputs}")
print(f"Outputs: {household_solved.outputs}")

Inputs: ['w', 'r', 'beta', 'psil', 'Tax', 'taul', 'gamma', 'H', 'sig_y', 'nh', 'thetay', 'thetao']
Outputs: ['ay', 'cy', 'ly', 'lo', 'lifebud_o', 'lifebud_y', 'co_y', 'co']


The lifetime budget blocks take the lifetime budget of each age and solve for co, co_y and the household block takes these solutions and returns cy, as well as labor for each age (ly, lo) and savings of the young (ay). I then combine them into a solved block.

In [6]:

@simple
def targets(ay,K,pi_y,pi_o,ly,lo,L,NFA):
    asset_mrkt  = pi_y*ay - K(+1)*NFA       
    labor_mrkt = pi_y*ly + pi_o*lo - L
        
    return asset_mrkt, labor_mrkt

@simple
def fiscal(taul,L,w):
    Tax = taul*L*w*2
    return Tax

        
OLGmodel = create_model([firm,fiscal,household_solved,targets],name='OLG')

print(f"Inputs: {OLGmodel.inputs}")
print(f"Outputs: {OLGmodel.outputs}")

unknowns_ss={'psil':.3,'beta':.967}
targets_ss={'labor_mrkt':0,'asset_mrkt':0}
ss = OLGmodel.solve_steady_state(calibration, unknowns_ss, targets_ss, solver="hybr")


Inputs: ['K', 'L', 'Z', 'alpha', 'delta', 'taul', 'beta', 'psil', 'gamma', 'H', 'sig_y', 'nh', 'thetay', 'thetao', 'pi_y', 'pi_o', 'NFA']
Outputs: ['r', 'w', 'Y', 'Tax', 'ay', 'cy', 'ly', 'lo', 'lifebud_o', 'lifebud_y', 'co_y', 'co', 'asset_mrkt', 'labor_mrkt']


In [7]:
inputs = ['taul']
unknowns = ['K','L']
targets = ['asset_mkt','labor_mrkt']

G = OLGmodel.solve_jacobian(ss, unknowns, targets, inputs, T=4)

LinAlgError: Last 2 dimensions of the array must be square

...but then I keep getting the following error message. Do you have a sense of what is happening? I've calculated what I think is the correct H_U matrix and it seems invertible/square with full dimension. 